In [ ]:
!pip install gradio openai pandas numpy scikit-learn


In [ ]:
import os

os.environ["OPENAI_API_KEY"] = input("Enter your OpenAI API key: ").strip()

if os.environ["OPENAI_API_KEY"]:
    print("✅ API key set")
else:
    print("⚠️ API key is empty!")


In [ ]:
import os
import gradio as gr
from openai import OpenAI

# -----------------------------
# OpenAI client
# -----------------------------
api_key = os.environ.get("OPENAI_API_KEY")
if not api_key:
    raise ValueError("OPENAI_API_KEY is not set. Please set the key before launching.")

client = OpenAI(api_key=api_key)

# -----------------------------
# 1. Social Media Content Agent
# -----------------------------
def generate_social_post(brand, platform, goal, tone, offer, extra):
    prompt = f"""
You are a professional social media content creator.

Brand: {brand}
Platform: {platform}
Goal: {goal}
Tone: {tone}
Offer: {offer or "None"}
Extra details: {extra or "None"}

Generate:
1. A catchy hook (1 line)
2. A main caption (3–5 short lines)
3. 8–12 relevant hashtags
4. A short call-to-action

Format your answer cleanly.
"""
    try:
        resp = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[{"role":"user","content":prompt}],
            temperature=0.8
        )
        return resp.choices[0].message.content
    except Exception as e:
        return f"Error calling OpenAI API:\n{e}"

def generate_content_plan(brand, platform, goal, days=7):
    prompt = f"""
Create a {days}-day social media content plan.

Brand: {brand}
Platform: {platform}
Goal: {goal}

For each day include:
- Theme/Idea
- Recommended format (post/reel/story/carousel)
- 1-line description

Return as bullet points, 1 per day.
"""
    try:
        resp = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[{"role":"user","content":prompt}],
            temperature=0.7
        )
        return resp.choices[0].message.content
    except Exception as e:
        return f"Error calling OpenAI API:\n{e}"

# -----------------------------
# 2. Build Gradio UI
# -----------------------------
with gr.Blocks(theme="soft") as demo:
    gr.Markdown("# 📣 Social Media Content Generator")
    gr.Markdown("Create high-performing captions & content plans instantly!")

    with gr.Tab("✨ Generate Post"):
        with gr.Row():
            brand = gr.Textbox(label="Brand Name", value="ShopSmart")
            platform = gr.Dropdown(
                choices=["Instagram", "Facebook", "LinkedIn", "Twitter/X", "YouTube"],
                value="Instagram",
                label="Platform"
            )

        with gr.Row():
            tone = gr.Dropdown(
                choices=["Friendly", "Professional", "Bold", "Fun", "Educational"],
                value="Friendly",
                label="Tone"
            )
            goal = gr.Textbox(label="Campaign Goal", value="Increase engagement for our festive sale.")

        offer = gr.Textbox(label="Offer (optional)", value="Flat 30% OFF on selected items.")
        extra = gr.Textbox(label="Extra Details (optional)", placeholder="Target audience, product focus, etc.")

        output_post = gr.Textbox(label="Generated Post", lines=10)
        btn_post = gr.Button("Generate Post")
        btn_post.click(generate_social_post,
                       inputs=[brand, platform, goal, tone, offer, extra],
                       outputs=output_post)

    with gr.Tab("📅 7-Day Content Plan"):
        brand2 = gr.Textbox(label="Brand Name", value="ShopSmart")
        platform2 = gr.Dropdown(
            choices=["Instagram", "Facebook", "LinkedIn", "Twitter/X", "YouTube"],
            value="Instagram",
            label="Platform"
        )
        goal2 = gr.Textbox(label="Campaign Goal", value="Grow brand visibility.")

        output_plan = gr.Textbox(label="7-Day Plan", lines=12)
        btn_plan = gr.Button("Generate 7-Day Content Plan")

        btn_plan.click(generate_content_plan,
                       inputs=[brand2, platform2, goal2],
                       outputs=output_plan)

demo.launch(share=True)
